In [ ]:
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns
from scipy import stats
import tensorboard as tb

In [ ]:
experiment_id = "MY5y2vC6SWuRQRQwXy83ZA"
experiment = tb.data.experimental.ExperimentFromDev(experiment_id)
experiment

In [ ]:
df = experiment.get_scalars()

In [ ]:
print(df["run"].unique())
print(df["tag"].unique())

In [ ]:
dfw = df.pivot_table(
        values="value",
        index=["run", "step"],
        columns="tag",
        dropna=False,
    )
dfw = dfw.reset_index()
dfw.columns.name = None
dfw.columns.names = [None for name in dfw.columns.names]

In [ ]:
print(dfw["run"].unique())

In [ ]:
dfw["step"] = dfw["step"] * dfw["epoch"].max() / dfw["step"].max()

In [ ]:
df_scale_1 = dfw[dfw["run"].str.endswith("00.0-1.0/version_0") | dfw["run"].str.endswith("00.0-1.0/version_1") | dfw["run"].str.endswith("00.0-1.0/version_2")]

In [ ]:
print(df_scale_1["run"].unique())

In [ ]:
method_split = df_scale_1.run.apply(lambda run: f"{run.split('-')[1]}".upper())
method_split.name = None

In [ ]:
sns.set(font_scale=1  , style="white")
plt.figure(figsize=(3,3))
plot = sns.lineplot(data=df_scale_1, x="step", y="valid_acc", hue=method_split, style=method_split, color='black')
plot.set(title=" ", xlabel="Trained epochs", ylabel="Validation accuracy", )
plot.axes.spines['top'].set_visible(False)
plot.axes.spines['right'].set_visible(False)
# plot.set_ylabel(None)
plt.tight_layout()
plt.savefig('hsd_accuracy.svg')

In [ ]:
df_multi_scale = dfw[dfw["run"].str.endswith("00.0-1.0/version_0") | dfw["run"].str.endswith("00.0-0.1/version_0") | dfw["run"].str.endswith("00.0-0.01/version_0")]
print(df_multi_scale["run"].unique())

In [ ]:
exodus_df_1 = df_multi_scale[df_multi_scale["run"] == "hsd-exodus-100000.0-1.0/version_0"]
exodus_df_01 = df_multi_scale[df_multi_scale["run"] == "hsd-exodus-100000.0-0.1/version_0"]
exodus_df_001 = df_multi_scale[df_multi_scale["run"] == "hsd-exodus-100000.0-0.01/version_0"]
slayer_df_1 = df_multi_scale[df_multi_scale["run"] == "hsd-slayer-100000.0-1.0/version_0"]
slayer_df_01 = df_multi_scale[df_multi_scale["run"] == "hsd-slayer-100000.0-0.1/version_0"]
slayer_df_001 = df_multi_scale[df_multi_scale["run"] == "hsd-slayer-100000.0-0.01/version_0"]

gdf = pd.DataFrame({
    'algorithm': ['EXODUS', 'EXODUS', 'EXODUS', 'EXODUS', 'SLAYER', 'SLAYER', 'SLAYER', 'SLAYER', 'EXODUS', 'EXODUS', 'EXODUS', 'EXODUS', 'SLAYER', 'SLAYER', 'SLAYER', 'SLAYER', 'EXODUS', 'EXODUS', 'EXODUS', 'EXODUS',   'SLAYER', 'SLAYER', 'SLAYER', 'SLAYER'],
    'Layer': [1, 2, 3, 4, 1, 2, 3, 4, 1, 2, 3, 4, 1, 2, 3, 4, 1, 2, 3, 4, 1, 2, 3, 4],
    'Gradient scaling': [1, 1, 1, 1, 1, 1, 1, 1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01],
    'Mean 2-norm': [
        exodus_df_1["grad_2.0_norm/network.0.0.weight_epoch"].mean(),
        exodus_df_1["grad_2.0_norm/network.1.0.weight_epoch"].mean(),
        exodus_df_1["grad_2.0_norm/network.2.0.weight_epoch"].mean(),
        exodus_df_1["grad_2.0_norm/network.3.weight_epoch"].mean(),
        slayer_df_1["grad_2.0_norm/linear_input.weight_epoch"].mean(),
        slayer_df_1["grad_2.0_norm/linear_hidden.0.weight_epoch"].mean(),
        slayer_df_1["grad_2.0_norm/linear_hidden.1.weight_epoch"].mean(),
        slayer_df_1["grad_2.0_norm/linear_output.weight_epoch"].mean(),
        exodus_df_01["grad_2.0_norm/network.0.0.weight_epoch"].mean(),
        exodus_df_01["grad_2.0_norm/network.1.0.weight_epoch"].mean(),
        exodus_df_01["grad_2.0_norm/network.2.0.weight_epoch"].mean(),
        exodus_df_01["grad_2.0_norm/network.3.weight_epoch"].mean(),
        slayer_df_01["grad_2.0_norm/linear_input.weight_epoch"].mean(),
        slayer_df_01["grad_2.0_norm/linear_hidden.0.weight_epoch"].mean(),
        slayer_df_01["grad_2.0_norm/linear_hidden.1.weight_epoch"].mean(),
        slayer_df_01["grad_2.0_norm/linear_output.weight_epoch"].mean(),
        exodus_df_001["grad_2.0_norm/network.0.0.weight_epoch"].mean(),
        exodus_df_001["grad_2.0_norm/network.1.0.weight_epoch"].mean(),
        exodus_df_001["grad_2.0_norm/network.2.0.weight_epoch"].mean(),
        exodus_df_001["grad_2.0_norm/network.3.weight_epoch"].mean(),
        slayer_df_001["grad_2.0_norm/linear_input.weight_epoch"].mean(),
        slayer_df_001["grad_2.0_norm/linear_hidden.0.weight_epoch"].mean(),
        slayer_df_001["grad_2.0_norm/linear_hidden.1.weight_epoch"].mean(),
        slayer_df_001["grad_2.0_norm/linear_output.weight_epoch"].mean(),
        ]
})
# gdf

In [ ]:
sns.set(font_scale=1, style="white")
hue_kw_args = {"marker" : ["v", "X", "o",]}
g = sns.FacetGrid(gdf, col="algorithm", hue="Gradient scaling", hue_kws=hue_kw_args, height=3, aspect=0.81, legend_out=False)
g.map(plt.plot, "Layer", "Mean 2-norm", linestyle="-", markersize=10, color='black')
g.set_titles('{col_name}')
axes = g.axes[0]
axes[0].legend(title="Surrogate scale")
axes[0].set_ylabel("Mean gradient 2-norm")
for line in axes[0].get_lines():
    line.set_color('C0')
for line in axes[1].get_lines():
    line.set_color("C1")
plt.yscale("log")
plt.tight_layout()
plt.savefig(f"hsd_mean_gradnorm.svg")